In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import random

plt.rcParams['image.cmap'] = 'gray'
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['figure.figsize'] = (12, 12)

In [ ]:
def load_acc(expdir):
    acc = np.load(expdir + '/accuracy.npy')
    return acc[acc>0]
def load_loss(expdir):
    acc = np.load(expdir + '/train_loss.npy')
    return acc[acc>0]

### Dating Classification Problem
Visualize training curves for learning a model to predict the date of a yearbook portrait.

In [ ]:
expdir0 = '../output/EXP' # inputs 96x96 

plt.rcParams['figure.figsize'] = (20, 8)
plt.figure()


plt.subplot(1,2,1)
plt.title('Validation Accuracy')
t1, = plt.plot(load_acc(expdir0))


plt.axis([0, 60, 0, .2])

plt.subplot(1,2,2)
plt.title('Train Loss')
m1, = plt.plot(load_loss(expdir0))
plt.axis([0, 60, 0, 5])
plt.legend([t1], ['DESCRIPTION'])


### Run the network on the validation set

In [ ]:
import caffe

# Config
caffe.set_device(1)
caffe.set_mode_gpu()
exp = 'EXP'
it = ITERATION

# load net
proto = '../output/{}/val.prototxt'.format(exp)
weights = '../output/{}/snapshots/train_iter_{}.caffemodel'.format(exp, it)
net = caffe.Net(proto, weights, caffe.TEST)

In [ ]:
import os
import numpy as np

data_dir = '../data/faces/women'
women_test = open(os.path.join(data_dir, 'test.txt')).read().splitlines()
preds = np.zeros(len(women_test))
gts = np.zeros(len(women_test))
diffs = np.zeros(len(women_test))
correct = 0
for i, path in enumerate(women_test):
    net.forward()
    prob_vec = np.squeeze(net.blobs['yrbook'].data)
    preds[i] = np.argmax(prob_vec)
    gts[i] = net.blobs['label'].data
    if preds[i] == gts[i]:
        correct += 1
    diffs[i] = np.abs(preds[i] - gts[i])
print(correct)
print(len(women_test))

### Confusion Matrix

In [ ]:
# Make a truncated colormap for easier visualization
import matplotlib.colors as colors
def truncate_colormap(cmap, minval=0.0, maxval=1.0, n=100):
    new_cmap = colors.LinearSegmentedColormap.from_list(
        'trunc({n},{a:.2f},{b:.2f})'.format(n=cmap.name, a=minval, b=maxval),
        cmap(np.append(np.array([0]),np.linspace(0.3, 1, 10))))
    return new_cmap
truncGreys = truncate_colormap(plt.cm.Greys, minval=0.3, maxval=1.0)
print(np.append(np.array([0]),np.linspace(0.3, 1, 10)))

In [ ]:
from sklearn.metrics import confusion_matrix
import matplotlib as mpl
import matplotlib.ticker as plticker
import matplotlib.dates as mdates
import seaborn as sns
import math

plt.rcParams['figure.figsize'] = (8,8)

plt.rcdefaults()
sns.set_style("white")
sns.set_style("ticks", {"xtick.major.size": 8, "ytick.major.size": 8, 'xtick.direction': u'in', 'ytick.direction': u'in'})

confmat = confusion_matrix(gts, preds)

confmat = confmat.astype('float') / confmat.sum(axis=1)[:, np.newaxis] # normalize
confmat[np.isnan(confmat)] = 0 


fig, ax = plt.subplots()
img = plt.imshow(confmat, interpolation='nearest', cmap=truncGreys) 
plt.tight_layout()
plt.ylabel('True Year', labelpad=8)
plt.xlabel('Predicted Year', labelpad=8)
cbar = plt.colorbar(img, fraction=0.046, pad=0.04)
cbar.set_label('Normalized Counts', labelpad=8, fontsize=18) 

new_tick_locs = [num for num in range(0,90, 10)]
def class_to_year(classes):
    return [str(c + 1928) for c in classes]
plt.yticks(new_tick_locs, class_to_year(new_tick_locs))
plt.xticks(new_tick_locs, class_to_year(new_tick_locs), rotation=45)
for item in ([ax.title, ax.yaxis.label, ax.xaxis.label]):
    item.set_fontsize(20)
    item.fontweight='bold'
for item in ax.get_xticklabels() + ax.get_yticklabels():
    item.set_fontsize(18)
plt.savefig('yrbook_confmat.pdf', bbox_inches='tight')

### Some simple metrics

In [ ]:
print 'Val Accuracy', correct / float(len(women_test))
print 'Avg Error', np.average(diffs)
print 'Median Error', np.median(diffs)
print 'Class. Accuracy Within 5 Years', len([d for d in diffs if d <= 5]) / float(len(women_test))
print 'Class. Accuracy Within 10 Years', len([d for d in diffs if d <= 10]) / float(len(women_test))